In [25]:
from langchain_community.llms import Ollama
import os
from dotenv import load_dotenv

load_dotenv()
OLLAMA_HOST=os.getenv('OLLAMA_HOST')

llm = Ollama(model='llama3', base_url=OLLAMA_HOST)

In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Based on the given databse schema below, write a SQL query that would answer the user's question:
    
    {schema}

    Question: {question}
    SQL Query:
    """
)


In [27]:
from langchain_community.utilities import SQLDatabase
db_url = 'sqlite:///chinook.db'

db = SQLDatabase.from_uri(db_url)

def get_schema(_):
    return db.get_table_info()

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop="\nSQL Query:")
    | StrOutputParser()
)

In [30]:
llm.invoke("who bult taj mahal?")

"The Taj Mahal is a mausoleum located in Agra, India, and it was built by Mughal Emperor Shah Jahan as a tribute to his beloved wife, Mumtaz Mahal. The construction of the Taj Mahal began in 1632 and took around 22 years to complete, with thousands of workers and artisans laboring to create this magnificent monument.\n\nMumtaz Mahal was Shah Jahan's favorite wife, whom he married in 1607. She was a Persian princess who was known for her beauty, kindness, and intelligence. Mumtaz Mahal died in childbirth in 1631, leaving Shah Jahan devastated. He decided to build the Taj Mahal as a mausoleum for her, using some of the finest white marble and other materials available at the time.\n\nThe Taj Mahal was designed by a team of architects led by Ustad Ahmad Lahori, who was a renowned architect of the time. The monument's design is a blend of Indian, Persian, and Islamic architectural styles, with intricate carvings, inlays, and calligraphy adorning its walls and domes.\n\nShah Jahan himself d

In [33]:
# create a vector store for the database schema
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
ollama_embeddings = OllamaEmbeddings(base_url="https://5763-34-142-189-205.ngrok-free.app")

schema = db.get_table_info()

vector_store = FAISS.from_texts(schema, ollama_embeddings)

vector_store.save_local("schema.faiss")

KeyboardInterrupt: 

In [29]:
sql_chain.invoke({"question": "What is the total number of tracks in the database?"})

ValueError: Ollama call failed with status code 500. Details: {"error":"option \"stop\" must be of type array"}